In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import random
import matplotlib.pyplot as plt

import skimage as sk
from skimage import io
from skimage import feature
from skimage import filters
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

from google.colab import files 
from google.colab import drive 

from PIL import Image
from math import sqrt

# WRITE CODE FOR IMAGES TO BE PULLED OFF GITHUB

In [ ]:
my_img = # CODE to pull from github

In [ ]:
# PVN IMAGE
#img_pvn1 = Image.open('my_path') #Rutuja use this one
img = sk.io.imread(myimg) #I changed this by cutting out Research folder this is for mac
image_gray = rgb2gray(img)
plt.imshow(image_gray)

# Find all Blobs and Label Training Data

#### REFINE THIS SO IT's JUST BLOBS DOG

In [ ]:
#SAMPLE IMAGE

# Compute radii in the 3rd column.
#blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
#threshold=.1
blobs_log_pvn = blob_log(image_gray, min_sigma=15, max_sigma=50, num_sigma=10, threshold=.1)
blobs_log_pvn[:, 2] = blobs_log_pvn[:, 2] * sqrt(2)

blobs_dog_pvn = blob_dog(image_gray, min_sigma=15, max_sigma=50, threshold=.1)
blobs_dog_pvn[:, 2] = blobs_dog_pvn[:, 2] * sqrt(2)

blobs_doh_pvn = blob_doh(image_gray, min_sigma=15, max_sigma=50, threshold=.1)
blobs_doh_pvn[:, 2] = blobs_doh_pvn[:, 2] * sqrt(2)

blobs_list = [blobs_log_pvn, blobs_dog_pvn, blobs_doh_pvn]
colors = ['lime', 'lime', 'lime']
titles = ['PVN LOG', 'PVN DOG', 'PVN DOH']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image_gray)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
len(blobs_dog_pvn)

#### CHANGE BELOW TO IGNORE IT AND STORE IT (EDGES AND CORNERS)

In [ ]:
# This section of code finds all the blobs and puts a 70x70 pixel square around them 
# then stores them in a list 

bounding_box_list = []

for blob in blobs_dog_pvn:
  x, y, r, = blob 
  y = int(y.round(0))
  x = int(x.round(0))
  maxx, maxy = image_gray.shape
  maxx-=1
  maxy-=1 #0-indexing
  left_bound = x-70
  right_bound = x+70
  upper_bound = y+70
  lower_bound = y-70
  if x < 70:
    left_bound = 0
    right_bound = 140-x
  if x > maxx-70:
    right_bound = maxx
    left_bound = 140 - (maxx-x)
  if y < 70:
    lower_bound = 0
    upper_bound = 140-y
  if y > maxy-70:
    lower_bound = maxy
    upper_bound = 140 - (maxy-y)

  bounding_box = image_gray[left_bound:right_bound,lower_bound:upper_bound]
  #Ignore this code. Just a temp solution to the images not being the same size
  #if(len(bounding_box) != 140):
  #  continue;
  #if(len(bounding_box[0]) != 140):
  #  continue;
  bounding_box_list.append(bounding_box) 


In [ ]:
# let's grab one cell as an example
plt.imshow(bounding_box_list[190])
bounding_box_list[190].shape

In [ ]:
# we need a training set, so let's grab random items from the entire blob list 
training_images = random.sample(bounding_box_list,10)

In [ ]:
test_images = random.sample(bounding_box_list,10) # test set 

In [ ]:
# now we need to label our training data, press 'c' for cell, or any other key for other.

# matplob inline DOESNT WORK in colaboratory (of course), so this needs to be downloaded and run as ipynb I think... 
# in order to include the user input 

%matplotlib inline

training_labels = []


def press(event):
    global keypress
    global vals
        
    if event.key == 'c':
        training_labels.append('cell')
    else: 
        training_labels.append('other')
    

    if len(training_labels) == 2:
        fig.canvas.mpl_disconnect(cid)

    return training_labels


for blobss in range(10):
    
    fig = plt.figure()
    plt.imshow(training_images[blobss])
    cid = fig.canvas.mpl_connect('key_press_event', press)


In [ ]:
# SAME PROCESS EXCEPT FOR TESTING DATA

%matplotlib inline

testing_labels = []


def press(event):
    global keypress
    global vals
        
    if event.key == 'c':
        testing_labels.append('cell')
    else: 
        testing_labels.append('other')
    

    if len(testing_labels) == 2:
        fig.canvas.mpl_disconnect(cid)

    return testing_labels


for blobss in range(10):
    
    fig = plt.figure()
    plt.imshow(test_images[blobss])
    cid = fig.canvas.mpl_connect('key_press_event', press)

In [ ]:
# let's check our labels
print(len(test_images))               
print(len(training_images))
print(len(training_labels))
print(len(testing_labels))

#### ADD IN ACCURACY / STORING INTO CSV or XCEL CODE 

# Begin keras code

### what categories do we want to use? should be decided before selecting training images

In [ ]:
# need to set pixel values to a 0-1 scale
train_images = train_images / 255.0

test_images = test_images / 255.0

#### What are the best layers to use? 

In [ ]:
#Load a previously saved model
#checkpoint_path will be a path to the checkpoint from the current directory, ie 'training_test/cp.ckpt'
def load_weights(model,checkpoint_path):
  model.load_weights(checkpoint_path)

In [ ]:
#Build model(if model has not already been built/fitted)
def build_model():
  # Need to build the model, why 128??
  model = keras.Sequential([
      keras.layers.Flatten(input_shape=(140, 140)),
      keras.layers.Dense(128, activation='relu'), # WHAT OTHER LAYERS SHOULD WE BE ADDING??
      keras.layers.Dropout(0.5),
      keras.layers.Dense(3)
  ])
  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model

In [ ]:
#Add training data to model
#Params: 
#model: the model to add to
#checkpoint_path: user defined path in case we would like to revert to this version
#training_images: training images
#training_labels: training labels
#epochs: number of epochs to use
def add_to_model(model, checkpoint_path, training_images, training_labels, num_epochs):
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
  model.fit(training_images, 
          training_labels,  
          epochs=num_epochs,
          callbacks=[cp_callback]) 
  

### code that calls these above functions as an example

In [ ]:
# OR USE THIS Proof-of-Concept


 # Need to build the model, why 128??
model = keras.Sequential([
      keras.layers.Flatten(input_shape=(140, 140)),
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(3)
  ])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(training_images, 
          training_labels,  
          epochs=300) #might lead to overfitting if training data is small


  

In [ ]:
# make predictions 

probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(test_images)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0]) #try a prediction out [notice how test labels are not involved]

In [ ]:
test_labels[0] #try a prediction out [notice how test labels are not involved]

# Plot results

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(1))
  plt.yticks([])
  thisplot = plt.bar(range(1), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
#https://www.tensorflow.org/tutorials/keras/classification
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 7
num_cols = 1
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()